In [1]:
asciiProposition = "=( >(A,B), |( ~(A) ,B) )"

## PReader
I couldn't make the PReader class be just a recursive algorithm. For long strings, this would mean that the stack would get very big. Some sort of buffer would have to prevent the stack from getting too big while also keeping the right order of characters. 

I created a char array from the string and a cursor. The cursor is a position indicator for which character is being read. In addition we would have to check for certain trigger characters, like operators for starting a new operation, parentheses to capture the operation in, and letters that would be variables. Taking this into account, we should also create a stack for read operations up until the current character. We would then interpret the operator when a closing parenthesis occurs.

In [2]:
from proposition import *
from proposition_parsing import *
from proposition_export import *
from hashing import *
from truthtable import *

prop = CompoundProposition(Operator.OR,
    CompoundProposition(Operator.OR, Variable('A'), Variable('B')),
    Variable('C')
)

print(prop)

|(|(A,B),C)


In [3]:
truthtable = TruthTable(prop)

print(truthtable)

A B C |(|(A,B),C)
0 0 0 0
0 0 1 1
0 1 0 1
0 1 1 1
1 0 0 1
1 0 1 1
1 1 0 1
1 1 1 1



In [4]:
simplified_truthtable = truthtable.simplify()

print(simplified_truthtable)

A B C |(|(A,B),C)
0 0 0 0
* * 1 1
* 1 * 1
1 * * 1



In [5]:
print(simplified_truthtable.get_binary_string())
print(simplified_truthtable.get_hash())

E
